---

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Oracle_logo.svg/2560px-Oracle_logo.svg.png" width="200" align = "left"></p>

# **<h1 align ="middle"><b> Oracle CloudWorld - Las Vegas</b></h1>**

### **<h1 align ="middle"><b> Use case 1. Person Detection in Video</b></h1>**

---

## **Prerequisites to run notebook**
- Perform all steps in the "1_prep_ocw_las_vegas_v1.ipynb" file
- Download folder from Github, containing 1) notebook, profile image example, VGG weights (vgg_face_weights.h5), score.py, and runtime.yaml
- Set up Custom Networking (......)
- Dynamic Group has been created and OCI Data Science policies are in order. Documentation can be found [here](https://docs.oracle.com/en-us/iaas/data-science/using/policies.htm)
- A correct config file with private key are stored in the /home/datascience/.oci directory

# **| 1. Import libraries**

In [1]:
import os
import numpy as np
import pandas as pd 
from deepface import DeepFace
import uuid
import glob      
import ocifs
import base64
import io
import matplotlib.pyplot as plt
import cv2
import fsspec
from PIL import Image
from io import BytesIO
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
from pytube import YouTube
import uuid

---

# **| 2. End to End Script**

### **There are several functions in the end to end script. They are splitted in:**
- **Function 1:** Function 1 fetches the YouTube URL, downloads the YouTube Video and stores it locally
- **Function 2:** Function 2 fetches the encoded profile image, decodes the profile images, and stores it locally
- **Prep for Predict:** Prep for Predict prepares for the predict function in deleting all previously ran files (e.g., images)
- **Predict:** Predict calls Function 1 and Function 2. Following, the function splits the video into images and runs DeepFace algorithm on the split images. DeepFace compares each splitted image with the profile image and reviews whether the same person is in both the profile image and the split image. 

In [2]:
import os
import numpy as np
import pandas as pd 
from deepface import DeepFace
import uuid
import glob      
import ocifs
import base64
import io
import matplotlib.pyplot as plt
import cv2
import fsspec
from PIL import Image
from io import BytesIO
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
from pytube import YouTube

##########################################################################################################################################
######################################################## Function 1          #############################################################
##########################################################################################################################################

def input_youtube_video(input_url):
    
    #delete previous videos
    !rm -r /home/datascience/youtube_videos

    #create a local directory to store the video
    path_input_locally = "/home/datascience/youtube_videos/" 

    try:       
        if not os.path.exists(path_input_locally):         
            os.makedirs(path_input_locally)    

    except OSError: 
        print ('Error: Creating directory for youtube video locally')
        

    #download file from youtube
    yt = YouTube(input_url)

    #store in local folder
    stream = yt.streams.get_by_itag(22)
    file_name_random = str(uuid.uuid4())
    file_location_local = stream.download(output_path=path_input_locally, filename  = file_name_random + ".mp4")
    
    print("Youtube download completed and stored in " + str(file_location_local))
    
    return file_location_local

##########################################################################################################################################
######################################################## Function 2          #############################################################
##########################################################################################################################################

def input_profile_image(profile_image_as_bytes):
    
    #create a local directory to store the image
    path_input_locally_image = "/home/datascience/profile_image/" 

    try:       
        if not os.path.exists(path_input_locally_image):         
            os.makedirs(path_input_locally_image)    

    except OSError: 
        print ('Error: Creating directory for profile image locally')
    
    ##### decoding of profile image
    img_bytes_p = io.BytesIO(base64.b64decode(profile_image_as_bytes.encode('utf-8')))
    profile_image = Image.open(img_bytes_p).resize((224, 224)) 
    
    #save image locally  
    profile_image_loc = path_input_locally_image + "pf_image.jpg"
    profile_image = profile_image.save(profile_image_loc)
    
    return profile_image_loc


##########################################################################################################################################
######################################################## Prep for Predict    #############################################################
##########################################################################################################################################

# Delete images if there are images in the local folder already
path_split_images = "/home/datascience/split_images"
files = glob.glob('/home/datascience/split_images/*.jpg')

for f in files:
    os.remove(f)

 #create a local folder to the images
path_split_images = "/home/datascience/split_images"

try:       
    # creating a folder named split_images 
    if not os.path.exists(path_split_images):         
        os.makedirs(path_split_images)    

except OSError: 
    print ('Error: Creating directory of data for split images')


##########################################################################################################################################
######################################################## Predict             #############################################################
##########################################################################################################################################

def predict(input_youtube_url, input_pf):
    
#     #fetch variables from data payload
#     profile_image_as_bytes = data['data']['pf_image']
#     input_url = data['data']['input_url']
    
    ######
    ###### Function 1    
    file_location_local = input_youtube_video(input_youtube_url)
    
    ######
    ###### Function 2
    #profile_image_loc = input_profile_image(profile_image_as_bytes) 
    #save image
    im1 = input_pf.save("./inputimage.jpg")
    
        
    print("Fetching video from " + file_location_local)
    #print("Fetching profile image from " + profile_image_loc)
    
    
    # Read the video from specified path 
    cam = cv2.VideoCapture(file_location_local)
    
    #get fps of original video
    fps = cam.get(cv2.CAP_PROP_FPS)
    print("**************************************************************** Original fps in video is " + str(fps))
    
    #define list of frames to analyze. 
    list_of_frames = list(range(1, 18001, int(fps)))  #starts at frame 1, ends at frame 601 (which is 20 seconds at FPS = 30 and 10 seconds at FPS = 60) with 30 frames in between (= 1 second). So, takes 20 (= 21 seconds) frames from the video

    #loop through the video and cut into images
    currentframe = 0

    while(True):
        
        for frame in list_of_frames:
            cam.set(cv2.CAP_PROP_POS_FRAMES, frame)
            print("Analyze frame number " + str(frame))

            # reading from frame 
            ret,frame = cam.read()

            if ret:
                if currentframe < 10:   
                    name = path_split_images + '/frame000' + str(currentframe) + '.jpg'           

                elif currentframe >= 10 and currentframe < 100:   
                    name = path_split_images + '/frame00' + str(currentframe) + '.jpg'          

                elif currentframe >= 100 and currentframe < 1000:   
                    name = path_split_images + '/frame0' + str(currentframe) + '.jpg'   

                else:
                    name = path_split_images + '/frame' + str(currentframe) + '.jpg'      

                print ('Creating...' + name) 

                # writing the extracted images 
                cv2.imwrite(name, frame) 

                # increasing counter
                currentframe += 1
            
        else: 
            break

    cam.release()    
    
    #apply DeepFace to the images
    try:
        dfs = DeepFace.find(img_path = "./inputimage.jpg", db_path = "/home/datascience/split_images", enforce_detection=False)  #first input is the profile image, second is the folder containing the split images
        
    except:
        pass #if no face is found in any of the images
    
    #get the dataframe of the results
    output_df = dfs[0]
    
    ########## calculations
    seconds_in_screen = output_df.shape[0]  # = total frames detected = frames per second as we are looping through each frame
    total_seconds_video_analyzed = len(list_of_frames)
    
    print("**************************************************************** Total seconds analyzed of entire video " + str(len(list_of_frames)) + " seconds")
    print("**************************************************************** This person was " + str(seconds_in_screen) + " seconds in screen")

    #delete pickle file and deleting the video afterwards
    !rm -r /home/datascience/split_images/representations_vgg_face.pkl
    
    in_screen = str()
    
    if seconds_in_screen > 0:
        in_screen = "Person is found in video"
    else:
        in_screen = "Person is not found in video"
        
    
    return in_screen, total_seconds_video_analyzed, seconds_in_screen
    

---

# **| 3. Testing the Script**

## **| 3.1 Input for Test**

In [3]:
# ############
# ############ Input 1 = input_url
# ############ 
# input_url = "https://www.youtube.com/shorts/ugwUcdtygok" 
# #other example: https://www.youtube.com/shorts/Y-PBRyEz4xY

# ############
# ############ Input 2 = Encoded Profile image
# ############ 

# #get a dummy example. In the video of Jimmy Car, we'll use a Jimmy Car Image to analyze the video.
# file_path="./jimmycar.jpg" 

# #open and encode the dummy image
# with open(file_path, "rb") as image2string:
#     converted_string = base64.b64encode(image2string.read()).decode('ascii')
               
# #add payload to full string
# payload1 = json.dumps(converted_string)
# pf_image_encoded = json.loads(payload1)

# ############
# ############ Input 1 and 2 into one payload
# ############ 
# data = {'data':{'pf_image': pf_image_encoded, 'input_url': input_url}}

## **| 3.2 Run Prediction in Gradio**

## examples
- Harvard lecture, 10 min video: https://www.youtube.com/watch?v=p7iwXvBnbIE

In [ ]:
import gradio as gr

with gr.Blocks() as demo:

    gr.Markdown("Detecting a Person in a (YouTube) video")
        
    with gr.Tab("YouTube Video"):
        with gr.Row():
            #input
            input_youtube_url = gr.Text(label="YouTube URL", info="Please add the full YouTube URL")
            input_pf = gr.Image(type='pil', label="Profile Image", info="Please add Profile Image here")
        
        #trigger
        button_1 = gr.Button("Cross-check Profile Image with the YouTube Video")
        
        ##output
        in_screen = gr.Text(label='Was person in screen?')
        total_seconds_video_analyzed = gr.Text(label='Total duration video in seconds')
        seconds_in_screen  = gr.Text(label='Total seconds person in profile image was in the video')

        
        ## buttons
        button_1.click(predict, inputs=[input_youtube_url, input_pf], outputs=[in_screen, total_seconds_video_analyzed, seconds_in_screen])

demo.launch(share=True, debug=True) #width=800, height=1100, 

Finding representations:  82%|████████▏ | 233/283 [00:48<00:10,  4.71it/s]

Running on local URL:  http://127.0.0.1:7860


Finding representations: 100%|█████████▉| 282/283 [00:59<00:00,  4.12it/s]

Running on public URL: https://34cd8298b288e7c816.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Finding representations: 100%|██████████| 283/283 [00:59<00:00,  4.73it/s]


Representations stored in /home/datascience/split_images/representations_vgg_face.pkl file.Please delete this file when you add new identities in your database.
find function lasts  60.730356216430664  seconds
**************************************************************** Total seconds analyzed of entire video 783 seconds
**************************************************************** This person was 111 seconds in screen


---

# **The End**